In [65]:
#Start Redis and MySQL servers
!docker compose up -d

[+] Running 0/0
 ⠋ Network ipvo_default  Creating                                          0.1s 
[+] Running 1/1
 ✔ Network ipvo_default    Created                                         0.1s 
 ⠋ Container redis-server  Creating                                        0.0s 
 ⠋ Container ipvo-db-1     Creating                                        0.0s 
[+] Running 1/3
 ✔ Network ipvo_default    Created                                         0.1s 
 ⠙ Container redis-server  Creating                                        0.1s 
 ⠙ Container ipvo-db-1     Creating                                        0.1s 
[+] Running 1/3
 ✔ Network ipvo_default    Created                                         0.1s 
 ⠿ Container redis-server  Starting                                        0.2s 
 ⠿ Container ipvo-db-1     Starting                                        0.2s 
[+] Running 1/3
 ✔ Network ipvo_default    Created                                         0.1s 
 ⠿ Container redis-server  St

In [ ]:
#Install python packages we'll be using if you don't have them
!python3 pip install mysql.connector
!python3 pip install redis
!python3 pip install faker

In [66]:
#Connecting to the MySQL server
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="user",
  password="password",
  database="db"
)

mycursor = mydb.cursor()

In [67]:
#Connecting to the Redis server
import redis

r = redis.Redis(host='localhost', port=6379, decode_responses=True)

In [68]:
#Importing Faker which is used for generating fake data, we need this because we're benchmarking writes
from faker import Faker
Faker.seed(123)
fake = Faker()

In [41]:
#Setting up how many writes/reads we want, 11 means we'll get 10 writes/reads
repeat = 11

In [42]:
#Checking if everything works on a small amount of data. This block writes data into cache(Redis) and the db(MySQL)
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
for key in range(1, repeat):
    name = fake.name()
    print(name)
    r.set(key, name)
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)

Brandon Russell
Steven Johnson
Evelyn Christian
George Cook
Aaron Graham
Kyle Jones
Jerome Whitehead
Charles Tyler
Thomas Berry
Ryan Mack


In [43]:
#Reading the data in cache and db to see if they were succesfully written
for key in range(1, repeat):
    temp = r.get(key)
    print(temp)
    sql = "SELECT value FROM tablica WHERE id = %s"
    tpl = (key, )
    mycursor.execute(sql, tpl)
    myresult = mycursor.fetchall()
    print(myresult)

Brandon Russell
[('Brandon Russell',)]
Steven Johnson
[('Steven Johnson',)]
Evelyn Christian
[('Evelyn Christian',)]
George Cook
[('George Cook',)]
Aaron Graham
[('Aaron Graham',)]
Kyle Jones
[('Kyle Jones',)]
Jerome Whitehead
[('Jerome Whitehead',)]
Charles Tyler
[('Charles Tyler',)]
Thomas Berry
[('Thomas Berry',)]
Ryan Mack
[('Ryan Mack',)]


In [44]:
#Reseting cache and db for the benchmark
r.flushall()
mycursor.execute("DROP TABLE tablica")

In [45]:
#Importing time, we need it to time how long it takes to execute writes/reads
import time

In [46]:
#Setting repeat to the number of reads/writes we want the benchmark to do
repeat = 101

In [47]:
#This block benchmarks how long it takes to write data if we don't have a cache once
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
t0 = time.time()
for key in range(1, repeat):
    name = fake.name()
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
t1 = time.time()
print("Time it took to write: ", t1-t0)

Time it took to write:  0.03865456581115723


In [48]:
#Reseting cache and db for the benchmark
r.flushall()
mycursor.execute("DROP TABLE tablica")

In [49]:
#This block benchmarks how long it takes to write data if we don't have a cache 100 times and calculates the average write time
ta = 0
for i in range(100):
    mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
    t0 = time.time()
    for key in range(1, repeat):
        name = fake.name()
        sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
        val = (key, name)
        mycursor.execute(sql, val)
    t1 = time.time()
    ta = ta + (t1 - t0)
    mycursor.execute("DROP TABLE tablica")
ta = ta/100
print("Average write time after repeating the benchmark 100 times is: ", ta)

Average write time after repeating the benchmark 100 times is:  0.17429688692092896


In [50]:
#This block benchmarks how long it takes to write data if we have a cache once
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
t0 = time.time()
for key in range(1, repeat):
    name = fake.name()
    r.set(key, name)
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
t1 = time.time()
print("Time it took to write: ", t1-t0)

Time it took to write:  0.4578871726989746


In [51]:
#Reseting cache and db for the benchmark
r.flushall()
mycursor.execute("DROP TABLE tablica")

In [52]:
#This block benchmarks how long it takes to write data if we have a cache 100 times and calculates the average write time
ta = 0
for i in range(100):
    mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
    t0 = time.time()
    for key in range(1, repeat):
        name = fake.name()
        r.set(key, name)
        sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
        val = (key, name)
        mycursor.execute(sql, val)
    t1 = time.time()
    ta = ta + (t1 - t0)
    r.flushall()
    mycursor.execute("DROP TABLE tablica")
ta = ta/100
print("Average write time after repeating the benchmark 100 times is: ", ta)


Average write time after repeating the benchmark 100 times is:  0.40509087562561036


In [53]:
#Benchmarking how long it takes to read data without a cache once
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
for key in range(1, repeat):
    name = fake.name()
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
t0 = time.time()
for key in range(1, repeat):
    sql = "SELECT value FROM tablica WHERE id = %s"
    tpl = (key, )
    mycursor.execute(sql, tpl)
    myresult = mycursor.fetchall()
    print(myresult)
t1 = time.time()
print("Time it took to write: ", t1-t0)
mycursor.execute("DROP TABLE tablica")

[('Matthew Adams',)]
[('Emily Fowler',)]
[('David Mayo',)]
[('Daniel Howard',)]
[('Allison Scott',)]
[('Michael Estes',)]
[('Anthony Martinez',)]
[('Lisa Smith',)]
[('Timothy Haney',)]
[('Tiffany Christensen',)]
[('Danielle Johnson',)]
[('Denise Robles',)]
[('Karina Davies',)]
[('Victoria Lewis',)]
[('Kristine Davis',)]
[('Kristi Cunningham',)]
[('Jonathan Campbell',)]
[('Ashley Parker',)]
[('Maria Booker',)]
[('Tanya Moore',)]
[('Carla Boyer',)]
[('Elizabeth Nunez',)]
[('Jack Mora',)]
[('William Adams',)]
[('Caleb Sullivan',)]
[('Sarah Myers',)]
[('Alexander Edwards',)]
[('Brittany Walker MD',)]
[('Scott Humphrey',)]
[('Donald Wright',)]
[('Randy Gamble',)]
[('Stephen Douglas',)]
[('Jonathan Johnson',)]
[('Juan Santiago',)]
[('Alexis Mason',)]
[('Teresa Mcdowell',)]
[('David Herrera',)]
[('Grace Hanson',)]
[('Michael Miller',)]
[('Charles Gordon',)]
[('Mr. Sean Stephens Jr.',)]
[('Whitney Huber',)]
[('Scott Patrick',)]
[('Christopher Castro',)]
[('Thomas Nixon',)]
[('Dr. Andrew Rodrig

In [54]:
#Benchmarking how long it takes to read data without a cache 100 times and calculating the average read time
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
for key in range(1, repeat):
    name = fake.name()
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
ta = 0
for i in range(100):
    t0 = time.time()
    for key in range(1, repeat):
        sql = "SELECT value FROM tablica WHERE id = %s"
        tpl = (key, )
        mycursor.execute(sql, tpl)
        myresult = mycursor.fetchall()
        print(myresult)
    t1 = time.time()
    ta = ta + (t1 - t0)
ta = ta/100
print("Average write time after repeating the benchmark 100 times is: ", ta)
mycursor.execute("DROP TABLE tablica")

[('Laura Martinez',)]
[('John Martin',)]
[('Travis Miller',)]
[('Jason Young',)]
[('Patricia Murphy',)]
[('Kyle White',)]
[('Katherine Wilkinson',)]
[('Steven Hahn',)]
[('Kristin Wilson',)]
[('Cindy Lucas',)]
[('Dr. Janice Robinson',)]
[('Mr. Kenneth Shelton',)]
[('Angela Morris',)]
[('Trevor Smith',)]
[('Cindy Yu',)]
[('Thomas Long',)]
[('Meagan Graham',)]
[('David Smith',)]
[('Jacob Brown',)]
[('Jocelyn Bell',)]
[('John Brown',)]
[('Michael Hubbard',)]
[('Lindsay Jones',)]
[('Ryan Griffin',)]
[('Denise Elliott',)]
[('Julie Cruz',)]
[('Christopher Gonzales',)]
[('Jean Johnson',)]
[('Mark Barajas',)]
[('Gregory Brennan',)]
[('Maurice Thomas',)]
[('Katherine Harper',)]
[('Richard Brown',)]
[('Rodney Hurst Jr.',)]
[('Brian Schmidt',)]
[('Isabella Phillips',)]
[('Vickie Sanchez',)]
[('Traci Nelson',)]
[('Deborah Martin',)]
[('Jeffrey Warner',)]
[('Mrs. Danielle Obrien DDS',)]
[('Angela Young',)]
[('Cindy Conner',)]
[('Jacob Hogan',)]
[('Allison Fields',)]
[('Jennifer Morales DVM',)]
[('Vi

In [55]:
#Benchmarking reads with cache hit rate 0% once
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
for key in range(1, repeat):
    name = fake.name()
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
t0 = time.time()
for key in range(1, repeat):
    temp = r.get(key)
    if temp == None:
        sql = "SELECT value FROM tablica WHERE id = %s"
        tpl = (key, )
        mycursor.execute(sql, tpl)
        myresult = mycursor.fetchall()
        print(myresult)
    else:
        print(temp)
t1 = time.time()
print("Time it took to read data: ", t1-t0)
r.flushall()
mycursor.execute("DROP TABLE tablica")

[('Vicki Ramirez',)]
[('Timothy Reyes',)]
[('Jeremy Rivera',)]
[('Tony Morgan',)]
[('Shannon Anderson',)]
[('Barbara Jenkins',)]
[('Amber Nichols',)]
[('Colleen Villa MD',)]
[('Cynthia Hall',)]
[('Shane Thompson',)]
[('Julia Myers',)]
[('Thomas Smith',)]
[('Dennis Doyle',)]
[('Dr. Jill Perez',)]
[('Alec Savage',)]
[('Erin Cooper',)]
[('Ashlee Jimenez',)]
[('Randy Nichols',)]
[('Robert Turner',)]
[('Caitlin Knight',)]
[('Carrie Robinson',)]
[('David Rose',)]
[('Jeremiah Dean',)]
[('David Barnett',)]
[('Angela Hudson',)]
[('Brittany Espinoza',)]
[('Kristin Perez',)]
[('Erin Miller',)]
[('Denise Leon',)]
[('Sean Bell',)]
[('Daniel Harrison',)]
[('Karen Erickson PhD',)]
[('Nicolas Mills',)]
[('James Martin',)]
[('Jessica Wheeler',)]
[('Elizabeth Bishop',)]
[('Annette White',)]
[('Julie Davis',)]
[('Glenn Dyer',)]
[('Melissa Foster',)]
[('Brandon Fuller',)]
[('Rachel Brewer',)]
[('Dr. Annette Murray',)]
[('David Smith',)]
[('Alexandra Dillon',)]
[('Jessica Greene',)]
[('Wendy Mccormick',)]


In [56]:
#Benchmarking reads with cache hit rate 0% 100 times and calculating average time it takes to execute reads
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
for key in range(1, repeat):
    name = fake.name()
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
ta = 0
for i in range(100):
    t0 = time.time()
    for key in range(1, repeat):
        temp = r.get(key)
        if temp == None:
            sql = "SELECT value FROM tablica WHERE id = %s"
            tpl = (key, )
            mycursor.execute(sql, tpl)
            myresult = mycursor.fetchall()
            print(myresult)
        else:
            print(temp)
    t1 = time.time()
    ta = ta + (t1 - t0)
ta = ta/100
print("Average read time after repeating the benchmark 100 times is: ", ta)
r.flushall()
mycursor.execute("DROP TABLE tablica")

[('Lisa Rocha',)]
[('Timothy Perez',)]
[('Steven Kerr',)]
[('James Warner',)]
[('Amber Black',)]
[('Michael Miller',)]
[('Daniel Robinson',)]
[('Chad Gibson',)]
[('Mrs. Carol Wood MD',)]
[('Wanda Beasley',)]
[('Rachel Keller',)]
[('Melanie Moore',)]
[('Renee Martinez',)]
[('Lisa Macias',)]
[('Joel Davis',)]
[('Tony Delacruz',)]
[('Tammy Tran',)]
[('Frederick Silva',)]
[('Pamela Crawford',)]
[('Corey Smith',)]
[('Kathryn Lowery',)]
[('Cory Walker',)]
[('Rose Baker',)]
[('Paula Strong',)]
[('Anna Humphrey',)]
[('Marco Morris',)]
[('Stefanie Clark',)]
[('Donna Barry',)]
[('Mathew Ortega MD',)]
[('Mary Nelson',)]
[('Veronica Hicks',)]
[('Travis Hill',)]
[('Lisa Roman',)]
[('Gary Martinez',)]
[('Harry Jensen',)]
[('Daniel Allen',)]
[('Terry White',)]
[('Angela Ballard',)]
[('Leslie Hart',)]
[('Benjamin Jones',)]
[('Brenda Fischer',)]
[('Donna Turner',)]
[('Amanda Brown',)]
[('Tamara Steele',)]
[('Ashley Foster',)]
[('Mark Price',)]
[('Amy Hampton',)]
[('Barry Vargas',)]
[('Justin Gould',)]


In [57]:
#Benchmarking reads with cache hit rate 50% once
cutoff = repeat * 0.5
cutoff = int(cutoff)
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
for key in range(1, cutoff):
    name = fake.name()
    r.set(key, name)
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
for key in range(cutoff, repeat):
    name = fake.name()
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
t0 = time.time()
for key in range(1, repeat):
    temp = r.get(key)
    if temp == None:
        sql = "SELECT value FROM tablica WHERE id = %s"
        tpl = (key, )
        mycursor.execute(sql, tpl)
        myresult = mycursor.fetchall()
        print(myresult)
    else:
        print(temp)
t1 = time.time()
print("Time it took to write: ", t1-t0)
r.flushall()
mycursor.execute("DROP TABLE tablica")

Charles Thornton
Amanda Harrison
Bryan Shaw
Grant Jones
Ashley James
Jeremy Delgado
Nicholas Hoffman
Cynthia Berg
John Cruz
Michelle Fisher
Mr. Edward Powell DDS
Morgan Miller
Eric Lindsey
John Neal
Courtney Berry
Rhonda Ali
John Hale
Heather Howard
Timothy Ramirez
Diane Mcbride
Sean Wilson
Ryan Gutierrez
Jerry Welch
Michael Faulkner
Rodney Crawford
Oscar White
Stacey Patton
Rachel Walton
Theresa Bell
Joshua Hall
James Ramirez
Roy Gibson
Yolanda Fletcher
Robin Mack
Kevin Bailey
Teresa Jacobs
Savannah Patterson
Sydney Hancock
Steven Long
Amy Cross
James Watson
Heather Brown
Tara Shaffer
Paul Lopez
Timothy Adams
Jessica Oconnell
Alexandra Wilson
Lori Tucker
Madison Jones
[('Teresa Carter',)]
[('Brooke Harvey',)]
[('Megan Chaney',)]
[('Michaela Proctor',)]
[('Lisa Brown',)]
[('Helen Reynolds',)]
[('Javier Schultz',)]
[('Troy Banks',)]
[('Rita Luna',)]
[('Bethany Moyer',)]
[('Cheyenne Reyes',)]
[('Hannah Martinez',)]
[('Michael Walker',)]
[('Joseph Banks',)]
[('Jacqueline Barrett',)]
[('Ch

In [58]:
#Benchmarking reads with cache hit rate 50% 100 times and calculating average read time
cutoff = repeat * 0.5
cutoff = int(cutoff)
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
for key in range(1, cutoff):
    name = fake.name()
    r.set(key, name)
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
for key in range(cutoff, repeat):
    name = fake.name()
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
ta = 0
for i in range(100):
    t0 = time.time()
    for key in range(1, repeat):
        temp = r.get(key)
        if temp == None:
            sql = "SELECT value FROM tablica WHERE id = %s"
            tpl = (key, )
            mycursor.execute(sql, tpl)
            myresult = mycursor.fetchall()
            print(myresult)
        else:
            print(temp)
    t1 = time.time()
    ta = ta + (t1 - t0)
ta = ta/100
print("Average read time after repeating the benchmark 100 times is: ", ta)
r.flushall()
mycursor.execute("DROP TABLE tablica")

Jonathan Nelson
Christy Robinson
Rachel Bolton
Amy Wolfe
Christine Ramirez
Michael Washington
Jamie Coleman
Gabriel Morgan
Dr. Christopher King MD
Caroline Dyer
Mary Smith
Scott Edwards
Paige Martin
John Smith
Nicole Grant
Gabrielle Reynolds
Shannon Williams
Tracy Bush
Kevin Robles
Julia Hogan
Marcus Vazquez
Anthony Weber
Jesus Barber
Raymond Foley
Emily Parrish
Sarah Reid
Steve Martin
Margaret Reid
Amber Pruitt
Richard Davis
Wesley Scott
Gerald Hess
Joseph Ochoa
Carmen May
Joshua Johnson
Kenneth White
Kellie Meadows
Barbara Daugherty
Caleb West DVM
Ricky Anderson
Ryan Ali
George Hall
Kevin Mcdonald
Autumn Clark MD
Jonathan Collins
Shawn Miranda
Lori Anderson
Matthew Holder
Joyce Hughes
[('Melissa Wallace',)]
[('Paige Mitchell',)]
[('Rebecca Murphy',)]
[('James James',)]
[('Jamie Perry',)]
[('Terri Park',)]
[('Thomas Gibson',)]
[('Emily Mendoza',)]
[('Jonathan Hatfield',)]
[('Corey Rice',)]
[('Katherine Clements',)]
[('Garrett Lang',)]
[('Jared Kaufman',)]
[('Raven Buchanan',)]
[('Adam

In [59]:
#Benchmarking reads with cache hit rate 80% once
cutoff = repeat * 0.8
cutoff = int(cutoff)
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
for key in range(1, cutoff):
    name = fake.name()
    r.set(key, name)
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
for key in range(cutoff, repeat):
    name = fake.name()
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
t0 = time.time()
for key in range(1, repeat):
    temp = r.get(key)
    if temp == None:
        sql = "SELECT value FROM tablica WHERE id = %s"
        tpl = (key, )
        mycursor.execute(sql, tpl)
        myresult = mycursor.fetchall()
        print(myresult)
    else:
        print(temp)
t1 = time.time()
print("Time it took to read: ", t1-t0)
r.flushall()
mycursor.execute("DROP TABLE tablica")

Kendra Flores
Danielle Morgan
Andrew Pierce
Beth Butler
Alejandro Anderson
Tammy Mcgrath
William Castro
Adam Kim
Sheila Harvey
Laurie Gonzales
Denise Rodriguez
Peter Perkins
Elizabeth Navarro
Tim Hunt
Alexander Taylor
Michael Jackson
Nicholas Brooks
Jennifer Hernandez
Dr. Julie Wang
Stephen Greer
Susan Knox
Kimberly Thompson
Hannah Choi
Katherine Page
Cynthia Smith
Gregory Alvarez
Marc Petty
Theresa Barrera
Scott Lindsey
Megan Espinoza
Cory Davis
Nicole Williamson
Donna Williams
Brian Carter
Anthony Garcia
Gilbert Carney
Amanda Henry
Terry Brown
Lori Stewart
Stanley Flores
Yvonne Hodges
Susan Green MD
Paul Dennis
Kimberly Reese
Christina Gamble
Scott Rogers
Jonathan Thompson
Glenn Crane
Deanna Brown
Taylor Evans
Elizabeth Gonzalez
Michael Warner
Rachel Medina
Amy Rangel
Jared Black
Cathy Gillespie
Linda Pitts
Jeffrey Spears
Daniel Watson
Veronica Henson
Jessica Thompson
Tyler Miller
David Collins
Chris Olsen
Alan Wright
Stephanie Martin
Angelica Stevens
Sandra Hill
Lindsay Kirby
Carrie

In [69]:
#Benchmarking reads with cache hit rate 80% 100 times and calculating average read time
cutoff = repeat * 0.8
cutoff = int(cutoff)
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
for key in range(1, cutoff):
    name = fake.name()
    r.set(key, name)
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
for key in range(cutoff, repeat):
    name = fake.name()
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
ta = 0
for i in range(100):
    t0 = time.time()
    for key in range(1, repeat):
        temp = r.get(key)
        if temp == None:
            sql = "SELECT value FROM tablica WHERE id = %s"
            tpl = (key, )
            mycursor.execute(sql, tpl)
            myresult = mycursor.fetchall()
            print(myresult)
        else:
            print(temp)
    t1 = time.time()
    ta = ta + (t1 - t0)
ta = ta/100
print("Average read time after repeating the benchmark 100 times is: ", ta)
r.flushall()
mycursor.execute("DROP TABLE tablica")

Brandon Russell
Steven Johnson
Evelyn Christian
George Cook
Aaron Graham
Kyle Jones
Jerome Whitehead
Charles Tyler
Thomas Berry
Ryan Mack
Robert Clark
Kelsey Pacheco
James Hammond
Melinda Little
Stefanie Tucker
Jade Johnson
Catherine Wilkins
Mary Jackson
James Jones
Kenneth Rogers
Brandon Fuller
Harold Perez
Misty Stout
Wendy Maynard
Cynthia Webb
William Wallace
Amanda Taylor
David Bell
Jeffrey Fisher
Arthur Kelly
Jeffrey Smith
Angel Benson
William Baker
Paul Rogers
Mark Booker
Jennifer Robbins
Jacqueline Young
Shawn Brown
Kimberly Thomas
Sydney Parsons
Cameron Proctor
Lori Pham
Antonio Taylor
Hannah Pitts
Veronica Barnett
Rick Johnson
Shelby Robertson
Jonathan Gardner
Steven Peterson
Jonathan Harris
James Mays
Craig Gonzalez
Stephanie Lee DDS
Evan Peterson
Tanya Lamb
Rachel Webb
Paula Scott
Laura Tucker
Danny Mason
Mark Villanueva
Lisa Simpson
Barbara Williams
Heidi Caldwell
Ian Townsend
Paul Hoover
William Brown
Jonathan Calderon
Justin Richardson
Barbara Baker
Erica Raymond
James Ro

In [61]:
#Benchmarking reads with cache hit rate 100% once
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
for key in range(1, repeat):
    name = fake.name()
    r.set(key, name)
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
t0 = time.time()
for key in range(1, repeat):
    temp = r.get(key)
    if temp == None:
        sql = "SELECT value FROM tablica WHERE id = %s"
        tpl = (key, )
        mycursor.execute(sql, tpl)
        myresult = mycursor.fetchall()
        print(myresult)
    else:
        print(temp)
t1 = time.time()
print("Time it took to write: ", t1-t0)
r.flushall()
mycursor.execute("DROP TABLE tablica")

Michelle Morrow
Christina Macias
Elijah Garner
Leon Torres
Kelly Greer
Bruce Richardson
Christine Sandoval
Alisha Jones
Yvonne Johnson
Richard Silva
Tiffany Arnold
Brady Weaver
John Delacruz
John Sullivan
Tiffany Rogers
Douglas Bailey MD
Jose Lopez
Diane Nichols
Audrey Bauer
Rita Olson
Mary Roy
Brendan Perez
Lauren Hughes
Christine Gibson
Linda Nguyen
Brittany Wheeler
Adriana Booker
Eduardo Mcdowell
Karen Johnston
Jessica Thomas
Michael Forbes
Jeffery Cain
Julie Guerra
Sheryl Little
Amanda Thompson
Jamie Moore
Phillip Santos
Danielle Spence
Jamie Harris
Steven Foley
Michael Collins
Carlos Castillo
Michael Howard
Mackenzie Smith
Latoya Brady
Melissa Meza
Stephanie Clements
Richard Chavez
Beverly Curry
Denise Holmes
Tasha Khan
Mitchell Gaines
Kaitlyn Jackson
Candice Davis
Alexander Edwards
Bradley Barnett
Justin Long
Shaun Perry
Jose Ryan
Ashley Lucas
Andrea Lopez
Nancy Wright
Jasmine Warner
Kimberly Reilly
Jonathan Torres
Charles Adams
Kevin Smith
Jason Miller
Francisco Parker
Drew Camp

In [62]:
#Benchmarking reads with cache hit rate 100% 100 times and calculating average read time
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
for key in range(1, repeat):
    name = fake.name()
    r.set(key, name)
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
ta = 0
for i in range(100):
    t0 = time.time()
    for key in range(1, repeat):
        temp = r.get(key)
        if temp == None:
            sql = "SELECT value FROM tablica WHERE id = %s"
            tpl = (key, )
            mycursor.execute(sql, tpl)
            myresult = mycursor.fetchall()
            print(myresult)
        else:
            print(temp)
    t1 = time.time()
    ta = ta + (t1 - t0)
ta = ta/100
print("Average read time after repeating the benchmark 100 times is: ", ta)
r.flushall()
mycursor.execute("DROP TABLE tablica")

Carla Haney
Jessica Cox
Tricia Bowman
Ricardo Randall
Emily Chan
Lorraine Valenzuela
Megan Huynh
Rebekah Davis
Manuel Ruiz
Jasmine Cooper
Debbie Nguyen
Jennifer Hamilton
Emily Allen
Brian Guzman
Andrea Schultz
Douglas Riley
Jessica Melendez
Mary Duffy
Blake Martinez
Amanda George
Hector Woodard
Joseph Rodgers
Deborah Rodriguez
Hunter Valdez
David Martinez
Jasmine Garcia
Barbara Fuller
Kevin Lawrence
Edwin Scott
Angela Snyder
Tracy Casey
Jennifer Hawkins
Jill Thomas
Nicholas Smith
Sean Shelton
George Jones
Regina Pacheco
Jamie Rivas
Mr. Brent Martinez
Barbara Randall
Joseph May
Mark Norris
Brittany Murphy
Terrence Garcia
Nicholas Nguyen
Michael Brady
Adam Oconnor
Justin Thomas
Charles Harvey
Brandi Gray
Kendra Nguyen
Paige Wong
Susan Davis
Mrs. Kathleen Brown
Michelle Miller
Yolanda Chaney
Daniel Rodriguez
Amber Tate
Theresa Mcgrath
Deborah Brown
Edward Nielsen
James Burton
Mrs. Lisa Sims
Cory Strickland
Katrina Roman
Tyrone Coleman
Eric Nguyen
Matthew Johnson
Patricia Williams
Jacob Ki

In [63]:
!docker compose down

[+] Running 0/0
 ⠋ Container ipvo-db-1     Stopping                                        0.1s 
 ⠋ Container redis-server  Stopping                                        0.1s 
[+] Running 0/2
 ⠙ Container ipvo-db-1     Stopping                                        0.2s 
 ⠙ Container redis-server  Stopping                                        0.2s 
[+] Running 0/2
 ⠹ Container ipvo-db-1     Stopping                                        0.3s 
 ⠹ Container redis-server  Stopping                                        0.3s 
[+] Running 0/2
 ⠸ Container ipvo-db-1     Stopping                                        0.4s 
 ⠸ Container redis-server  Stopping                                        0.4s 
[+] Running 0/2
 ⠼ Container ipvo-db-1     Stopping                                        0.5s 
 ⠼ Container redis-server  Stopping                                        0.5s 
[+] Running 0/2
 ⠴ Container ipvo-db-1     Stopping                                        0.6s 
 ⠴ Container 